In [8]:
#Parameters
g_output_category="Food"
g_output_calendar="month"


In [9]:
from graphviz import Digraph
import pandas as pd
g_Cat_Tree={} # child to parent mapping

allCategories_DF = pd.read_csv("Data/categories.csv")
G = Digraph()

for i in range(0, len(allCategories_DF)):
    G.edge(allCategories_DF["Parent"].iloc[i],allCategories_DF["Child"].iloc[i])
    g_Cat_Tree[allCategories_DF["Child"].iloc[i]]=allCategories_DF["Parent"].iloc[i]

G.render('bin/Categories.gv', view=True) 

'bin/Categories.gv.pdf'

In [10]:
import pandas as pd
import numpy as np

def getLevel(node, target, thisTree):
    level = 0
    try:
        while(node!=target):
            node = thisTree[node]
            level += 1
        return level;
    except:
        return -1
    
def getCategory(storeName, minLen):
    try:
        candidates_DF = g_allLocations.loc[storeName[:minLen]]
        shortListCandidatesSeries = []
        if (type(candidates_DF) is pd.Series):
            shortListCandidatesSeries.append(candidates_DF)
        else:
            for i in range(len(candidates_DF)):
                shortListCandidatesSeries.append(candidates_DF.iloc[i])
        
        for candidate in shortListCandidatesSeries:
            if (storeName.startswith(candidate.loc["Substring"])):
                return candidate["Category"]
        return "Invalid Transaction"
    except:
        return "Invalid Transaction"

# Load the data and some useful global variables
g_allLocations = pd.read_csv("Data/locations.csv")
g_allLocations_min_len = min([len(x) for x in g_allLocations["Substring"] ])
g_allLocations["FirstLetters"] = g_allLocations["Substring"].apply(lambda x:  x[:g_allLocations_min_len])
g_allLocations.set_index("FirstLetters" , inplace=True)

allTransactions = pd.read_csv("Data/2018.csv", quotechar='"', header=None, names=['Date','Store','Payed','Received',"CC"])
allTransactions['Payed']=pd.to_numeric(allTransactions['Payed'])

allTransactions["Category"] = allTransactions["Store"].apply(lambda x: getCategory(x, g_allLocations_min_len))
allTransactions["IncludeTransaction"] = allTransactions["Category"].apply(lambda x: getLevel(x,g_output_category, g_Cat_Tree))


relevantTransactions = allTransactions[allTransactions["IncludeTransaction"]>=0]

if g_output_calendar=='week':
    relevantTransactions["Class"]=pd.to_datetime(relevantTransactions["Date"]).apply(lambda x: x.date().isocalendar()[0:2])
elif g_output_calendar=='month':
    relevantTransactions["Class"]=pd.to_datetime(relevantTransactions["Date"]).apply(lambda x: (x.date().year,x.date().month))
elif g_output_calendar=='year':
    relevantTransactions["Class"]=pd.to_datetime(relevantTransactions["Date"]).apply(lambda x: x.date().year)

relevantTransactions.groupby(["Class", "Category"]).agg({"Payed":sum,"Received":sum})

C:\Users\Voicu\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,,Payed,Received
Class,Category,,
"(2017, 3)",Food,215.19,NaN
"(2017, 4)",Food,318.18,NaN
"(2017, 5)",Food,431.97,NaN
"(2017, 6)",Food,171.52,NaN
"(2017, 7)",Food,432.21,NaN
"(2017, 8)",Food,346.28,NaN
"(2017, 9)",Food,398.98,NaN
"(2017, 10)",Food,229.18,NaN
"(2017, 11)",Food,306.35,NaN
